# Integration Portal

# RELEASE 24 Readout

## Specialty Pharmacy Portal

-- Errors have occurred with the portal on both the IQVIA and the Deloitte development tracks.

-- R24 Deployment Errors: Code migration was not synchronized, the field mapping between systems did not go live with R24.

    -- This caused many records to error out. Records were still created in production on Monday and Tuesday.

    -- All records that were created in the system on Monday and Tuesday were deleted from production. The job for Wednesday was held.

    -- On Thursday, the mapping was corrected. All recieved records since launch were reloaded into the system as 'New'.

    -- After this fix was implemented, it was discovered that permissions issues were still present on the document+registration records. Permissions were only granted to the IT Admin and Integration users. Permissions are needed for FRM, Agent, and LPS Administrators. The following fields are not able to be seen in the system by the lower permission sets:
    
    •	Mandatory Transfer
    •	PA Not Available
    •	ME Submitted
    •	ME Submitted Date
    •	ME Outcome
    •	ME Denial Reason


-- IQVIA Data Quality Errors: Tabs were entered into string values on the inbound IQVIA file. This is causing errors to occur when loading records into the PCP. IQVIA is taking corrective action to prevent sending values with tabs present.


**Wednesday 5/5/2021** -- patches are being worked on, the issues have been raised and temporary processes are being set up to cover the errors.

![](https://theinflammatorydisease.files.wordpress.com/2019/10/specialty-pharmacy-1.jpg)



# Salesforce -- The Patient Connections Platform
## Customer Support Program Web-App

- Lilly customer support programs are designed to provide our patients with excellent customer service. Our goal is to get patients their prescriptions as fast as possible, and at an affordable price. 


![](https://www.salesforce.com/content/dam/web/en_is/www/images/home/unified/emea-astro-device.png)

# Simple-Salesforce
## PCP REST API

-- Automated processes require code in order to operate... this is the code
* automation can be written in any language... there are a lot of programming languages. I'm using python for NLP and Artificial Intelligence purposes. If you don't like python, I'm sorry you feel that way.


# Specialty Pharmacy Portal (IQVIA)

IQVIA develops a webapp that provides us with valueable data. We use that data for automated metric reporting... which is why I'm using a notebook. We are attempting to centralize our integrated data inside of the PCP so that we can track the patient journey according to their specific "Program"

## Establishing a connection to the server, must run before you attempt to run any other chunk of code!

This lets us establish an API connection into the PCP

In [2]:
# ------------------------------- #
# imports 
# ------------------------------- #

import os
from simple_salesforce import Salesforce

# defining our log in credentials, saved as environment variables
username = os.environ.get('FORCE_PROD_USER')
password = os.environ.get('FORCE_PROD_PW')
security_token = os.environ.get('FORCE_PROD_TOKEN')


# ------------------------------- #
# imports 
# ------------------------------- #

def connect_to_prod(use_proxies):
    """ proxies give us elevated security during our connections sessions """
    if use_proxies:
        proxies = {
            'http': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000',
            'https': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000'
        }

        sf = Salesforce(username=username, password=password, security_token=security_token, proxies=proxies)
    else:
        sf = Salesforce(username=username, password=password, security_token=security_token)
    return sf

# call the connection to see if we can access data
connect_to_prod(True)

## Setting up a function to write SOQL queries

This lets us do magic, don't worry about it


In [3]:
def query(sf, soql):
    # uses query_all so results are not paginated...
    soql_query = sf.query_all(soql)
    
    size = soql_query['totalSize']
    done = soql_query['done']
    # records are the actual data returned inside of the function's json
    records = soql_query['records']
    """ i am reformatting the return result so that this custom function just gives a clean python list """
    returned_records = []
    for record in records:
        record_fields = {}
        attributes = record['attributes']
        for key in record.keys():
            if record[key] != attributes:
                record_fields[key] = record[key]
        returned_records.append(record_fields)
    # cleaned python list of records, each record will have JSON to hold data
    return returned_records

# Monitor number of new documents that agents are queued up to process
Allows us to see health of the queue

In [4]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"

# return the results of the query in a list using our custom function
docs = query(sf, soql)

# print out the number of docs our query returned
print(len(docs))

7


# Create a script that allows us see the associated data
-- the sp file that is recieved every night is not cleaned before being loaded into the PCP, we just use a document + registration record and have agents manually process the document. For automation, we would look to clean the data by grouping the information according to the copay card and then creating a new "SP Status Update" record inside of the PCP. We would then connect that update into the associated program appropriately.

In [5]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map


def parse_copay_map(new_copay_cards_recieved):
    for card in new_copay_cards_recieved:
        print(card)


def clean_copay_map(copay_map):
    """ allows us to parse out the registrations and documents that are attached to a specific savings card id """
    first_card = list(copay_map.keys())[0]
    
    # allows us access to new registrations associated with the card... they aren't registrations, they are status updates
    registrations = copay_map[first_card][0]
    # documents are created in SFDC
    documents = copay_map[first_card][1]
    
    # cleaning the data to read as a status update (the way I loaded it in my django script...)
    for status_update in registrations:
        # show the data structure... its coverage
        hub_patient_id = status_update['Hub_Patient_ID__c']

        pa_submitted = status_update['PA_Submitted__c']
        pa_sumbitted_date = status_update['PA_Submitted_Date__c']
        pa_outcome = status_update['PA_Outcome__c']
        pa_denial_reason = status_update['PA_Denial_Reason__c']
        pa_expired = status_update['PA_Expired__c']
        appeal_submitted = status_update['Appeal_Submitted__c']
        appeal_submitted_date = status_update['Appeal_Submitted_Date__c']
        appeal_outcome = status_update['Appeal_Outcome__c']
        appeal_denial_reason = status_update['Appeal_Denial_Reason__c']
        
        benefit_converted = status_update['Benefit_Converted__c']
        
        date_action_recorded = status_update['Date_Action_Recorded__c']
        
        prescriber_first_name = status_update['DTPC_Prescriber_First_Name__c']
        prescriber_last_name = status_update['DTPC_Prescriber_Last_Name__c']
        prescriber_address = status_update['DTPC_Prescriber_Address__c']
        prescriber_address_2 = status_update['Prescriber_Address_2__c']
        prescriber_city = status_update['DTPC_Prescriber_City__c']
        prescriber_state = status_update['DTPC_Prescriber_State__c']
        prescriber_zip = status_update['Prescriber_Zip__c']
        prescriber_phone = status_update['DTPC_Prescriber_Phone__c']
        npi = status_update['NPI__c']
        
        copay_card_number = status_update['DTPC_Copay_Card_Number__c']
        
        insurnace_effective_date = status_update['DTPC_DocuSign_RxBIN__c']
        rx_bin = status_update['DTPC_DocuSign_RxBIN__c']
        rx_pcn = status_update['DTPC_DocuSign_RxPCN__c']
        ncpdp = status_update['NCPDP__c']
        rx_group = status_update['DTPC_DocuSign_RxGroup__c']
        primary_insurance_number = status_update['DTPC_Primary_Insurance_Number__c']
        primary_insurance_company_phone = status_update['DTPC_Primary_Insurance_Company_Phone__c']
        primary_insurance_company = status_update['DTPC_Primary_Insurance_Company__c']
        insurance_plan_name = status_update['DTPC_Insurance_Plan_Name__c']
        
        pharmacy_name = status_update['Pharmacy_Name__c']

        # every status update should have the same HUB patient ID, because we grouped the updates together...
        print(first_card, "\t", hub_patient_id, "\t", pharmacy_name)
        

# ------------------------------------------------------------------- #
# scripts in the django format require a 'run' similar to a 'main'
# ------------------------------------------------------------------- #

def run():
    test_map = group_documents_by_copay_id(docs)
    # parse_copay_map(test_map)
    clean_copay_map(test_map)


# run the script
run()


E46103092427 	 P-0000877415 	 SENDERRA
E46103092427 	 P-0000877415 	 SENDERRA
E46103092427 	 P-0000877415 	 SENDERRA


## Process based reporting

We are now going to use all of the functions that I just shared to show off how we can construct different types of reports that access data in the PCP. This specific notebook will monitor the inbound records recieved via IQVIA for the SP Portal.

## SP Portal

This process is going to analyze data that was loaded into salesforce from IQVIA, the data is manually processed by call center agents today... we are working on automating that...

In [6]:

# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map

test_map = group_documents_by_copay_id(docs)
print(type(test_map))

<class 'dict'>


## Evaluate New Inbound Copay Cards
*Simple function will print out all of the card ids that have recieved a new update*

This will evaluate all of the new cards that were recieved. We need to reconsider for the design of the system how we store and update SP Status updates. Python will allow us to cache information quickly before trying to process it into the PCP.

- Instead of doing what I am doing here: pass the file into Django via sFTP, pre-process the file so that updates are created according to savings card. DO THE GROUPING I AM SHOWING HERE, then create a table to store the associated update information (pull data from all related records in the same file).

- This will show you what we do today.... which is bad design... we load each row in the file as a "Document + Registration" record and then we are now trying to write automation off of that object design... which is already flawed and bad automation. 

In [7]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"



# return the results of the query in a list using our custom function
docs = query(sf, soql)


def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    
    # return the map
    return copay_card_map


def parse_copay_map(new_copay_cards_recieved):
    print(len(new_copay_cards_recieved))
    """
    for card in new_copay_cards_recieved:
        print(card)
    """
    

# ------------------------------------------------------------------- #
# scripts in the django format require a 'run' similar to a 'main'
# ------------------------------------------------------------------- #

def run():
    test_map = group_documents_by_copay_id(docs)
    parse_copay_map(test_map)


# run the script
run()

4


## Data Cleaning
*parse the new card values for update types*

Use a Linux server instead of informatica, build out in deldev3

## The PCP program object

The salesforce application that we are connected to is called the 'Patient Connections Platform'. This app is built off of a series of totally custom objects and workflows. The core object of the app is called the 'Program'.

A program record represents a single patient's enrollment into a Customer Support Program. I've created a bunch of utility functions that will outline the data model for a program and allow us to quickly query records in the PCP. Reading through this will help you understand the related objects to a CSP patient's program.

*remember, these functions are designed to abstract away complexity, i am showing off the usage of the simple-salesforce package through a notebook but structuring these functions through a file directory will allow you to make them reusable*

In [8]:
# --------------------------------- #
# consent parsing
# --------------------------------- #

def parse_consents(program):
    """ consents are processed upon patient enrollment into a CSP and stored to either a patient/caregiver through
     the program record """
    try:
        consents = program['Consent_Auth_Preferences__r']['records']
        consent_data = []
        for record in consents:
            consent_record = {}
            for key in record:
                if key != 'attributes':
                    consent_record[key] = record[key]
            consent_data.append(consent_record)
    except KeyError:
        consent_data = []

    # return a list of all the consent records cleaned up
    return consent_data


# --------------------------------- #
# coverage parsing
# --------------------------------- #

def parse_coverages(program):
    """ takes in a program and returns a list containing a list of copay coverages and a list of insurance coverages """
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = [[], []]
        coverages = program['Coverage1__r']['records']
        for record in coverages:
            try:
                if record['RecordTypeId'] == '012f20000001HL3AAM':
                    copay_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            copay_coverage_record[key] = record[key]
                    coverage_data[0].append(copay_coverage_record)

                elif record['RecordTypeId'] == '012f20000001HL4AAM':
                    insurance_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            insurance_coverage_record[key] = record[key]
                    coverage_data[1].append(insurance_coverage_record)

            except KeyError:
                print("queries that incorporate coverage records must include the RecordTypeId field")

    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = [[], []]
    return coverage_data


def filter_active_coverages(coverages):
    """ takes a list of coverage records and parses out any inactive records (good for separating ins and copay) """
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['DTPC_Record_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires you to pull the DTPC_Record_Status__c from coverage")
    return active_coverages


def parse_old_coverages(program):
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = []
        coverages = program['Coverage__r']['records']
        for record in coverages:
            old_coverage_record = {}
            for key in record:
                if key != 'attributes':
                    old_coverage_record[key] = record[key]
            coverage_data.append(old_coverage_record)
    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = []
    return coverage_data


def filter_active_old_coverages(coverages):
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['Coverage_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires your query to pull the Coverage_Status__c field from coverage")
    return active_coverages


# --------------------------------- #
# claims parsing
# --------------------------------- #

def parse_claims(program):
    try:
        claims = program['Claims_Adjudication__r']['records']
        claims_data = []
        for record in claims:
            claim_record = {}
            for key in record:
                if key !='attributes':
                    claim_record[key] = record[key]
            claims_data.append(claim_record)
    except KeyError:
        claims_data = []

    return claims_data


# --------------------------------- #
# dispense parsing
# --------------------------------- #

def parse_dispenses(program):
    try:
        dispenses = program['Specialty_Pharmacy_Dispenses__r']['records']
        dispense_data = []
        for record in dispenses:
            dispense_record = {}
            for key in record:
                if key != 'attributes':
                    dispense_record[key] = record[key]
            dispense_data.append(dispense_record)
    except KeyError:
        dispense_data = []

    return dispense_data


# --------------------------------- #
# program services parsing
# --------------------------------- #

def parse_program_services(program):
    try:
        services = program['Program_Services__r']['records']
        services_data = []
        for record in services:
            service_record = {}
            for key in record:
                if key != 'attributes':
                    service_record[key] = record[key]
            services_data.append(service_record)
    except KeyError:
        services_data = []

    return services_data


# --------------------------------- #
# agent workflow parsing
# --------------------------------- #

def parse_cases(program):
    try:
        cases = program['Cases__r']['records']
        case_data = []
        for record in cases:
            case_record = {}
            for key in record:
                if key != 'attributes':
                    case_record[key] = record[key]
            case_data.append(case_record)
    except KeyError:
        case_data = []

    return case_data


def parse_document_logs(program):
    logs = program['Document_Logs__r']
    try:
        document_logs = logs['records']
        document_data = []
        for record in document_logs:
            document_record = {}
            for key in record:
                if key != 'attributes':
                    document_record[key] = record[key]
            document_data.append(document_record)

    except KeyError:
        document_data = []

    # return the two separate data structures
    return document_data

# Cool Support Methods that will help you write better Automation
-- Don't think about it too hard...

## List queries

this support function will allow us to do more interesting queries... you'll see it in action shortly, but it will be allow us to write queries that take in a list of specific values and return data connected to that specific list of records. For example, if I want to look at 10 specific records and return related record data specific to only those 10 records, we would call this function.

In [9]:
def query_list(sf, soql, the_list):
    """ when using the IN operator, if the list contains more than 20k characters, it hits a limit so i'll partition the query using chunks of length 700 """
    records = []
    number_of_lists = round(len(the_list) / 700)
    if number_of_lists == 0:
        number_of_lists = 1
    start_index = 0
    try:
        partition_length = round(len(the_list) / number_of_lists)
    except:
        partition_length = round(len(the_list))
    end_index = partition_length
    for i in range(number_of_lists):
        sub_list = the_list[start_index:end_index]
        list_string = format_list(sub_list)
        sub_soql = soql + "%s" % list_string
        try:
            sf_data = query(sf, sub_soql)
        except ValueError:
            sf_data = []
            print("SOQL error, no data returned")
        for record in sf_data:
            records.append(record)
        start_index = end_index
        end_index = partition_length * (i + 2)
    # returning all the records found in the query
    return records